In [ ]:
# drAbreu/bc4chemd_ner

# ner_tags: Array of tags, where 0 indicates no disease mentioned, 1 signals the first token of a chemical and 2 the subsequent chemical tokens.

# tner/bc5cdr

# {
#     "O": 0,
#     "B-Chemical": 1,
#     "B-Disease": 2,
#     "I-Disease": 3,
#     "I-Chemical": 4
# }


# ncbi_disease


# id: Sentence identifier.
# tokens: Array of tokens composing a sentence.
# ner_tags: Array of tags, where 0 indicates no disease mentioned, 1 signals the first token of a disease and 2 the subsequent disease tokens.




In [ ]:
!pip install transformers seqeval datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 

**Data Links**

https://huggingface.co/datasets/tner/bc5cdr




In [ ]:
import numpy as np
import pandas as pd
import datasets

from transformers import AutoModelForTokenClassification, AutoTokenizer, DataCollatorForTokenClassification
import torch


In [ ]:
# bc4chemd = datasets.load_dataset("drAbreu/bc4chemd_ner")
bc5cdr = datasets.load_dataset("tner/bc5cdr")
# ncbi = datasets.load_dataset("ncbi_disease")
# bc5cdr2 = datasets.load_dataset("bigbio/bc5cdr")



  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(bc5cdr)


DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5228
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5330
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5865
    })
})


In [ ]:
bc5cdr["train"][70]

{'tokens': ['This',
  'toxicity',
  'appeared',
  'in',
  'patients',
  'receiving',
  'the',
  'higher',
  'doses',
  'of',
  'desferrioxamine',
  'or',
  'coincided',
  'with',
  'the',
  'normalization',
  'of',
  'ferritin',
  'or',
  'aluminium',
  'serum',
  'levels',
  '.'],
 'tags': [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]}

In [ ]:
# for x in bc5cdr["train"]:
#   for y in x["tags"]:
#     if y > 4:
#       print("found >", y) 
#       break



In [ ]:
print(bc5cdr["train"].features["tags"])

Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)


In [ ]:
label_list = ["O", "B-Chemical", "B-Disease", "I-Disease","I-Chemical"]

label_list

['O', 'B-Chemical', 'B-Disease', 'I-Disease', 'I-Chemical']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


# def tokenize_function(examples):
#     return tokenizer(examples["tokens"], padding="max_length", truncation=True,is_split_into_words=True)


# tokenized_datasets = bc5cdr.map(tokenize_function, batched=True)



In [ ]:
example = bc5cdr["train"][70]

example

{'tokens': ['This',
  'toxicity',
  'appeared',
  'in',
  'patients',
  'receiving',
  'the',
  'higher',
  'doses',
  'of',
  'desferrioxamine',
  'or',
  'coincided',
  'with',
  'the',
  'normalization',
  'of',
  'ferritin',
  'or',
  'aluminium',
  'serum',
  'levels',
  '.'],
 'tags': [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]}

In [ ]:
tokenized_input = tokenizer(example["tokens"],is_split_into_words=True)
tokenized_input

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'This',
 'toxicity',
 'appeared',
 'in',
 'patients',
 'receiving',
 'the',
 'higher',
 'doses',
 'of',
 'des',
 '##fer',
 '##rio',
 '##xa',
 '##mine',
 'or',
 'coincided',
 'with',
 'the',
 'normal',
 '##ization',
 'of',
 'f',
 '##er',
 '##rit',
 '##in',
 'or',
 'aluminium',
 'serum',
 'levels',
 '.',
 '[SEP]']

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 10, 11, 12, 13, 14, 15, 15, 16, 17, 17, 17, 17, 18, 19, 20, 21, 22, None]


In [ ]:

# Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and 
# the labels of all other tokens to the label of the word they come from. 
# Another strategy is to set the label only on the first token obtained from a given word, 
# and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, 
# just change the value of the following flag:

label_all_tokens = True



In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(bc5cdr['train'][:5])

{'input_ids': [[101, 11896, 2858, 21501, 1162, 7936, 1116, 1103, 2848, 7889, 17786, 5026, 2109, 2629, 1104, 172, 4934, 2386, 2042, 119, 102], [101, 1130, 8362, 6354, 2050, 4638, 26300, 117, 20061, 1193, 177, 24312, 27291, 13475, 1103, 9711, 1107, 1892, 2997, 1105, 1762, 2603, 1666, 1118, 1107, 4487, 7912, 2285, 172, 4934, 2386, 2042, 117, 126, 1106, 1406, 17599, 12139, 1116, 120, 4023, 117, 1108, 1107, 23034, 1174, 1137, 11802, 1118, 9468, 2858, 19315, 117, 121, 119, 102], [101, 123, 1106, 123, 17713, 120, 4023, 119, 102], [101, 1109, 177, 1183, 11439, 5026, 2109, 2629, 1104, 1620, 17713, 120, 4023, 11164, 118, 1899, 18873, 2572, 4163, 1108, 1145, 6320, 11802, 1118, 9468, 2858, 21501, 1162, 119, 102], [101, 11896, 2858, 21501, 1162, 2041, 1225, 1136, 6975, 1719, 1892, 2997, 1137, 1762, 2603, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
tokenized_bc5cdr = bc5cdr.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5228 [00:00<?, ? examples/s]

Map:   0%|          | 0/5330 [00:00<?, ? examples/s]

Map:   0%|          | 0/5865 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
import torch.cuda

if torch.cuda.is_available():
  print("GPU available")
  model = model.to("cuda")

GPU available


In [ ]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#finetuning the model. Defining the training arguments
from transformers import TrainingArguments, Trainer

batch_size = 16

args = TrainingArguments(
    "bert_base_cased_finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    # push_to_hub=True,
)



In [ ]:
# data collator that will batch our processed examples together 
# while applying padding to make them all the same size 
#  (each pad will be padded to the length of its longest example).

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
#load the seqeval metric
from datasets import load_metric

metric = load_metric("seqeval")

In [ ]:
# test if it is working–
labels = [label_list[i] for i in example["tags"]]
metric.compute(predictions=[labels], references=[labels])

{'Chemical': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'Disease': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:

import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_bc5cdr["train"],
    eval_dataset=tokenized_bc5cdr["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.159711,0.855798,0.891572,0.873319,0.946998
2,0.204600,0.167224,0.871495,0.906804,0.888799,0.949465
3,0.204600,0.178233,0.894477,0.893901,0.894189,0.953831
4,0.062700,0.202716,0.892983,0.908336,0.900594,0.955083
5,0.024100,0.240929,0.886570,0.913822,0.899989,0.955032
6,0.024100,0.254865,0.890304,0.908612,0.899364,0.954009
7,0.013700,0.262164,0.884709,0.917407,0.900761,0.954460
8,0.005800,0.281036,0.889376,0.913852,0.901448,0.954734


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.159711,0.855798,0.891572,0.873319,0.946998
2,0.204600,0.167224,0.871495,0.906804,0.888799,0.949465
3,0.204600,0.178233,0.894477,0.893901,0.894189,0.953831
4,0.062700,0.202716,0.892983,0.908336,0.900594,0.955083
5,0.024100,0.240929,0.886570,0.913822,0.899989,0.955032
6,0.024100,0.254865,0.890304,0.908612,0.899364,0.954009
7,0.013700,0.262164,0.884709,0.917407,0.900761,0.954460
8,0.005800,0.281036,0.889376,0.913852,0.901448,0.954734
9,0.005800,0.281733,0.898846,0.907386,0.903096,0.955719
10,0.003300,0.285647,0.894577,0.910971,0.902700,0.955560


TrainOutput(global_step=3270, training_loss=0.048224420394372504, metrics={'train_runtime': 1088.7021, 'train_samples_per_second': 48.02, 'train_steps_per_second': 3.004, 'total_flos': 2186525239292160.0, 'train_loss': 0.048224420394372504, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.28564712405204773,
 'eval_precision': 0.894576862886722,
 'eval_recall': 0.910971498620901,
 'eval_f1': 0.902699747942543,
 'eval_accuracy': 0.9555597927988051,
 'eval_runtime': 27.7527,
 'eval_samples_per_second': 192.054,
 'eval_steps_per_second': 12.035,
 'epoch': 10.0}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_bc5cdr["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("jang007/Bert_finetuned_bc5cdr")

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("jang007/Bert_finetuned_bc5cdr")
model.to("cuda")


In [ ]:
from transformers import AutoTokenizer
import torch
import numpy as np

label_list = ["O", "B-Chemical", "B-Disease", "I-Disease","I-Chemical"]
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# text = "he was sick and he took tylenol and tums. He was having head ache"

# text = """ 
# PREOPERATIVE DIAGNOSES:,  Hypertrophy of tonsils and adenoids, and also foreign body of right ear.,POSTOPERATIVE DIAGNOSES:,  Hypertrophy of tonsils and adenoids, and also foreign body of right ear.,OPERATIONS:,  Tonsillectomy, adenoidectomy, and removal of foreign body (rock) from right ear.,ANESTHESIA:,  General.,HISTORY: , The patient is 5-1/2 years old.  She is here this morning with her Mom.  She has very large tonsils and she snores at night and gets up frequently at night and does not sleep well.  At the office we saw the tonsils were very big.  There was a rock in the right ear and it was very deep in the canal, near the drum.  We will remove the foreign body under the same anesthetic.,PROCEDURE:,:  Natalie was placed under general anesthetic by the orotracheal route of administration, under Dr. XYZ and Ms. B.  I looked into the left ear under the microscope, took out a little wax and observed a normal eardrum.  On the right side, I took out some impacted wax and removed the rock with a large suction.  It was actually resting on the surface of the drum but had not scarred or damaged the drum.  The drum was intact with no evidence of middle ear fluid.  The microscope was set aside.  Afrin drops were placed in both nostrils.  The neck was gently extended and the Crowe-Davis mouth gag inserted.  The tonsils and adenoids were very large.  The uvula was intact.  Adenoidectomy was performed using the adenoid curette with a tonsil sponge placed into the nasopharynx.  Tonsillectomy accomplished by sharp and blunt dissection.  Hemostasis achieved with electrocautery and the tonsils beds injected with 0.25% Marcaine with 1:200,000 epinephrine.  Sutures of zero plain catgut next were used to re-approximate the posterior to the anterior tonsillar pillars, suturing these down to the tonsillar beds.  Sponge is removed from the nasopharynx.  The suction electrocautery was used for pinpoint hemostasis on the adenoid bed.  We made sure the cautery tip did not come into the contact with the soft palate or the eustachian tube orifices.  The nose and throat were then irrigated with saline and suctioned.  Excellent hemostasis was observed.  An orogastric tube was placed.  The stomach found to be empty.  The tube was removed, as was the mouth gag.  Sponge and needle count were reported correct.  The child was then awakened and prepared for her to return to the recovery room.  She tolerated the operation excellently.
# """

text = """
PAST MEDICAL HISTORY:,  Significant for hypertension.  The patient takes hydrochlorothiazide for this.  She also suffers from high cholesterol and takes Crestor.  She also has dry eyes and uses Restasis for this.  She denies liver disease, kidney disease, cirrhosis, hepatitis, diabetes mellitus, thyroid disease, bleeding disorders, prior DVT, HIV and gout.  She also denies cardiac disease and prior history of cancer.,PAST SURGICAL HISTORY: , Significant for tubal ligation in 1993.  She had a hysterectomy done in 2000 and a gallbladder resection done in 2002.,MEDICATIONS: , Crestor 20 mg p.o. daily, hydrochlorothiazide 20 mg p.o. daily, Veramist spray 27.5 mcg daily, Restasis twice a day and ibuprofen two to three times a day.,ALLERGIES TO MEDICATIONS: , Bactrim which causes a rash.  
"""


# inputs = tokenize_and_align_labels(text)
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to("cuda")

tokens = inputs.tokens()
print(tokens)

outputs = model(**inputs)
print(outputs)

predictions = torch.argmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()


# predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)


for token, prediction in zip(tokens, predictions[0]):
  if label_list[prediction] != 'O':
    print((token, label_list[prediction]))

['[CLS]', 'PA', '##ST', 'ME', '##DI', '##CA', '##L', 'H', '##IS', '##TO', '##R', '##Y', ':', ',', 'Sign', '##ificant', 'for', 'h', '##yper', '##tens', '##ion', '.', 'The', 'patient', 'takes', 'h', '##ydro', '##ch', '##lor', '##oth', '##ia', '##zi', '##de', 'for', 'this', '.', 'She', 'also', 'suffers', 'from', 'high', 'ch', '##ole', '##ster', '##ol', 'and', 'takes', 'C', '##rest', '##or', '.', 'She', 'also', 'has', 'dry', 'eyes', 'and', 'uses', 'Rest', '##asis', 'for', 'this', '.', 'She', 'denies', 'liver', 'disease', ',', 'kidney', 'disease', ',', 'c', '##ir', '##r', '##hos', '##is', ',', 'he', '##pa', '##titis', ',', 'diabetes', 'me', '##lli', '##tus', ',', 'thy', '##roid', 'disease', ',', 'bleeding', 'disorders', ',', 'prior', 'D', '##V', '##T', ',', 'HIV', 'and', 'go', '##ut', '.', 'She', 'also', 'denies', 'cardiac', 'disease', 'and', 'prior', 'history', 'of', 'cancer', '.', ',', 'PA', '##ST', 'S', '##UR', '##GI', '##CA', '##L', 'H', '##IS', '##TO', '##R', '##Y', ':', ',', 'Sign', '

In [ ]:
# #just trying out the pipeline function


# from transformers import pipeline

# from transformers import AutoModelForTokenClassification

# from transformers import AutoTokenizer
# import torch
# import numpy as np

# label_list = ["O", "B-Chemical", "B-Disease", "I-Disease","I-Chemical"]
# tokenizer2 = AutoTokenizer.from_pretrained("bert-base-cased")

# model2 = AutoModelForTokenClassification.from_pretrained("jang007/Bert_finetuned_bc5cdr")

# # text = "he was sick and he took tylenol and tums. He was having head ache"

# # text = """ 
# # PREOPERATIVE DIAGNOSES:,  Hypertrophy of tonsils and adenoids, and also foreign body of right ear.,POSTOPERATIVE DIAGNOSES:,  Hypertrophy of tonsils and adenoids, and also foreign body of right ear.,OPERATIONS:,  Tonsillectomy, adenoidectomy, and removal of foreign body (rock) from right ear.,ANESTHESIA:,  General.,HISTORY: , The patient is 5-1/2 years old.  She is here this morning with her Mom.  She has very large tonsils and she snores at night and gets up frequently at night and does not sleep well.  At the office we saw the tonsils were very big.  There was a rock in the right ear and it was very deep in the canal, near the drum.  We will remove the foreign body under the same anesthetic.,PROCEDURE:,:  Natalie was placed under general anesthetic by the orotracheal route of administration, under Dr. XYZ and Ms. B.  I looked into the left ear under the microscope, took out a little wax and observed a normal eardrum.  On the right side, I took out some impacted wax and removed the rock with a large suction.  It was actually resting on the surface of the drum but had not scarred or damaged the drum.  The drum was intact with no evidence of middle ear fluid.  The microscope was set aside.  Afrin drops were placed in both nostrils.  The neck was gently extended and the Crowe-Davis mouth gag inserted.  The tonsils and adenoids were very large.  The uvula was intact.  Adenoidectomy was performed using the adenoid curette with a tonsil sponge placed into the nasopharynx.  Tonsillectomy accomplished by sharp and blunt dissection.  Hemostasis achieved with electrocautery and the tonsils beds injected with 0.25% Marcaine with 1:200,000 epinephrine.  Sutures of zero plain catgut next were used to re-approximate the posterior to the anterior tonsillar pillars, suturing these down to the tonsillar beds.  Sponge is removed from the nasopharynx.  The suction electrocautery was used for pinpoint hemostasis on the adenoid bed.  We made sure the cautery tip did not come into the contact with the soft palate or the eustachian tube orifices.  The nose and throat were then irrigated with saline and suctioned.  Excellent hemostasis was observed.  An orogastric tube was placed.  The stomach found to be empty.  The tube was removed, as was the mouth gag.  Sponge and needle count were reported correct.  The child was then awakened and prepared for her to return to the recovery room.  She tolerated the operation excellently.
# # """

# text = """
# PAST MEDICAL HISTORY:,  Significant for hypertension.  The patient takes hydrochlorothiazide for this.  She also suffers from high cholesterol and takes Crestor.  She also has dry eyes and uses Restasis for this.  She denies liver disease, kidney disease, cirrhosis, hepatitis, diabetes mellitus, thyroid disease, bleeding disorders, prior DVT, HIV and gout.  She also denies cardiac disease and prior history of cancer.,PAST SURGICAL HISTORY: , Significant for tubal ligation in 1993.  She had a hysterectomy done in 2000 and a gallbladder resection done in 2002.,MEDICATIONS: , Crestor 20 mg p.o. daily, hydrochlorothiazide 20 mg p.o. daily, Veramist spray 27.5 mcg daily, Restasis twice a day and ibuprofen two to three times a day.,ALLERGIES TO MEDICATIONS: , Bactrim which causes a rash.  
# """



# ner =pipeline("ner", aggregation_strategy="simple",model=model2,tokenizer=tokenizer2)

# ner(text)
